In [732]:
import pandas as pd
import re
from IPython.display import Markdown, display
from collections import Counter
import networkx as nx

def periodize_dates(dates:list=[], delta:datetime.timedelta=timedelta(days=14), dateformat='%Y-%m-%d'):
    """https://gist.github.com/kallewesterling/9a8d12ce073776ed52865bfb362ad073"""

    try:
        dates = sorted([datetime.datetime.strptime(x, dateformat) for x in dates])
    except ValueError as e:
        date = re.search(r'''['"](.*)['"] does not match format''', str(e))
        if date:
            date = date.groups()[0]
        raise ValueError(f'A date found in list that did not adhere to format (`{date}`). Needs to follow format `{dateformat}`.') from None

    if isinstance(delta, int):
        delta = timedelta(days=delta)

    periods = []

    for ix, date in enumerate(dates):
        min_date = date - delta
        max_date = date + delta

        prev_date, next_date = None, None
        start_chain, end_chain, in_chain, solo_date = None, None, None, None
        prev_date_in_range, next_date_in_range = None, None

        try:
            if ix-1 >= 0:
                prev_date = dates[ix-1]
        except IndexError:
            prev_date = None

        try:
            next_date = dates[ix+1]
        except IndexError:
            next_date = None

        if next_date:
            next_date_in_range = next_date >= min_date and next_date <= max_date

        if prev_date:
            prev_date_in_range = prev_date >= min_date and prev_date <= max_date

        if all([next_date, prev_date, prev_date_in_range, next_date_in_range]):
            # In the loop and in a chain (near previous date and next)
            in_chain = True
        elif all([next_date, prev_date, next_date_in_range]) and not prev_date_in_range:
            # In the loop and beginning of a chain (not near previous date but near next)
            start_chain = True
        elif all([next_date, prev_date, prev_date_in_range]) and not next_date_in_range:
            # In the loop and end of a chain (near previous date but not next)
            end_chain = True
        elif all([next_date, prev_date]) and not all([prev_date_in_range, next_date_in_range]):
            # In the loop but solo date (not not near previous date nor next)
            solo_date = True
        elif next_date and next_date_in_range:
            # In the loop but solo date (not not near previous date nor next)
            start_chain = True
        elif next_date:
            solo_date = True
        elif prev_date and prev_date_in_range:
            end_chain = True
        elif prev_date:
            solo_date = True
        elif not next_date and not prev_date:
            solo_date = True
        else:
            raise RuntimeError('An unexpected error occurred.')

        if start_chain:
            periods.append([date])

        elif end_chain:
            periods[len(periods)-1].append(date)

        elif solo_date:
            periods.append([date])

        elif in_chain:
            periods[len(periods)-1].append(date)
            
    return periods


# Data clean up functions
def has_required_data(row):
    '''(internal) for use with DataFrame lambda function to ensure that any given row has the required data present'''
    has_performer = row['Performer'] != '' or row['Normalized performer'] != '' or (row['Performer first-name'] != '' or row['Performer last-name']) != ''
    # has_city = row['City'] or row['Normalized City']
    has_venue = row['Venue'] != ''
    if has_performer and has_venue:
        return True
    else:
        return False

def has_correct_date(row):
    '''(internal) for use with DataFrame lambda function to ensure that any given row has a correct date present'''
    return re.search(r'\d{4}\-\d{2}\-\d{2}', row['Date']) != None


def get_performer(row, null_value=''):
    '''(internal) for use with DataFrame lambda function to return the cleaned-up version of a performer's name (in an order of priority)'''

    if row['Performer first-name'] and row['Performer last-name']:
        return row['Performer first-name'] + ' ' + row['Performer last-name']
    
    for r in ['Normalized performer', 'Performer']
        if row[r]:
            return row[r]
    
    return null_value

    
def get_city(row, null_value=''):
    '''(internal) for use with DataFrame lambda function to return the cleaned-up version of a city's name (in an order of priority)'''
    for r in ['Normalized City', 'City']
        if row[r]:
            return row[r]
    
    return null_value


def get_unique_venue(row, null_value=''):
    '''(internal) for use with DataFrame lambda function to return the cleaned-up version of a venue's name (in an order of priority)'''
    if row['Venue'] and row['City']:
        return row['Venue'] + ' (' + row['City'] + ')'
    
    for r in ['Venue', 'City']
        if row[r]:
            return row[r]

    return null_value


def get_source(row, null_value=''):
    '''(internal) for use with DataFrame lambda function to return the cleaned-up version of a source (in an order of priority)'''
    for r in ['Source clean', 'Source']
        if row[r]:
            return row[r]

    return null_value


def get_revue(row, null_value=''):
    '''(internal) for use with DataFrame lambda function to return the cleaned-up version of a revue's name (in an order of priority)'''
    for r in ['Normalized Revue Name', 'Revue name']
        if row[r]:
            return row[r]
    
    return null_value

**7995 rows imported.**

In [735]:
# Fix basic stuff
df.replace('—', '', inplace=True)
df.replace('–', '', inplace=True)
df.fillna('', inplace=True)

In [736]:
# Filter
df.drop(df[df['Exclude from visualization'] == True].index, inplace=True)
Markdown(f'**{df.shape[0]} rows after filtering**: Exclusion from visulization.')

**7971 rows after filtering**: Exclusion from visulization.

In [737]:
# Filter
df.drop(df[df['Unsure whether drag artist'] == 'TRUE'].index, inplace=True)
Markdown(f'**{df.shape[0]} rows after filtering**: Unsure whether drag artist.')

**7494 rows after filtering**: Unsure whether drag artist.

In [738]:
df['has_required_data'] = df.apply(lambda row: has_required_data(row), axis=1)
df.drop(df[df['has_required_data'] == False].index, inplace=True)
Markdown(f'**{df.shape[0]} rows after filtering**: Required data.')

**5518 rows after filtering**: Required data.

In [739]:
df['has_correct_date'] = df.apply(lambda row: has_correct_date(row), axis=1)
df.drop(df[df['has_correct_date'] == False].index, inplace=True)
Markdown(f'**{df.shape[0]} rows after filtering**: Full date in `Date` column.')

**5456 rows after filtering**: Full date in `Date` column.

In [740]:
# Clean up names
df['Performer'] = df.apply(lambda row: get_performer(row), axis=1)
df['City'] = df.apply(lambda row: get_city(row), axis=1)
df['Source'] = df.apply(lambda row: get_source(row), axis=1)
df['Revue'] = df.apply(lambda row: get_revue(row), axis=1)
df['Unique venue'] = df.apply(lambda row: get_unique_venue(row), axis=1)
Markdown(f'_Cleaned up all names_.')

_Cleaned up all names_.

In [741]:
# Drop *node* information - i.e. not relevant for edges
node_information = {}
node = {
    'Legal name': None,
    'Alleged age': None,
    'Assumed birth year': None,
    'Source': None
}
for ix, row in df.iterrows():
    if row['Legal name'] or row['Alleged age'] or row['Assumed birth year']:
        if row['Performer'] not in node_information:
            node_information[row['Performer']] = []
        
        d = node
        for cat in ['Legal name', 'Assumed birth year', 'Alleged age', 'Source']:
            d[cat] = row[cat]
            
        node_information[row['Performer']].append(d)
Markdown(f'_Extracted node information_.')

_Extracted node information_.

In [742]:
# Drop unnecessary information
cols = ['Revue name', 'Normalized Revue Name', 'Legal name', 'Alleged age', 'Assumed birth year', 'Source clean', 'Category', '2020-12-31 ID', 'Normalized City', 'Performer first-name', 'Performer last-name', 'Normalized performer', 'has_required_data', 'has_correct_date', 'Exclude from visualization', 'Unsure whether drag artist']
for col in cols:
    try:
        del df[col]
    except KeyError:
        pass # already gone
Markdown(f'_Removed all unneccesary columns_:' + '\n - ' + '\n -'.join(cols))

_Removed all unneccesary columns_:
 - Revue name
 -Normalized Revue Name
 -Legal name
 -Alleged age
 -Assumed birth year
 -Source clean
 -Category
 -2020-12-31 ID
 -Normalized City
 -Performer first-name
 -Performer last-name
 -Normalized performer
 -has_required_data
 -has_correct_date
 -Exclude from visualization
 -Unsure whether drag artist

In [743]:
all_venues = list(set([x for x in df['Unique venue']]))

dfs = {}
for venue in [x for x in all_venues]:
    dfs[venue] = {'df': df[df['Unique venue'] == venue], 'count': 0}
    for col in dfs[venue]['df'].columns:
        if not col in ['Date', 'Performer', 'Unique venue', 'City', 'Source']:
            del dfs[venue]['df'][col]
    dfs[venue]['count'] = len(dfs[venue]['df'])
    dfs[venue]['all_dates'] = sorted(list(set([x for x in dfs[venue]['df']['Date']])))
    dfs[venue]['periodized_dates'] = periodize_dates(dfs[venue]['all_dates'], timedelta(days=14))

In [745]:
# df['venue-link'] = ''

In [746]:
venue_connections = {}
for venue in dfs:
    if not venue in venue_connections:
        venue_connections[venue] = {}
    for period, dates in enumerate(dfs[venue]['periodized_dates'], start=1):
        if not period in venue_connections[venue]:
            venue_connections[venue][period] = {'dates': [], 'performers': []}
        for date in dates:
            venue_connections[venue][period]['dates'].append(date.strftime('%Y-%m-%d'))
            for x, y in dfs[venue]['df'][dfs[venue]['df']['Date'] == date.strftime('%Y-%m-%d')].iterrows():
                venue_connections[venue][period]['performers'].append(y['Performer'])
        venue_connections[venue][period]['performers'] = list(set(venue_connections[venue][period]['performers']))
        venue_connections[venue][period]['dates'] = sorted(list(set(venue_connections[venue][period]['dates'])))

In [788]:
performer_connections = {}

for venue, venue_connection in venue_connections.items():
    for _, data in venue_connection.items():
        for performer_out in data['performers']:
            other_performers = [x for x in data['performers'] if not x == performer_out]
            weight = len(data['dates'])
            min_date = min([datetime.datetime.strptime(x, '%Y-%m-%d') for x in data['dates']])
            max_date = max([datetime.datetime.strptime(x, '%Y-%m-%d') for x in data['dates']])
            if not other_performers:
                pass # Only one performer appeared in the period
            for performer_in in other_performers:
                if not performer_out in performer_connections:
                    performer_connections[performer_out] = []
                performer_connections[performer_out].append((performer_in, weight, venue, min_date, max_date))

In [795]:
G = nx.MultiDiGraph()

for source, connections in performer_connections.items():
    for connection in connections:
        target, weight, venue, min_date, max_date = connection
        G.add_edges_from([(source, target, weight, {'venue': venue, 'min_date': min_date.strftime('%Y-%m-%d'), 'max_date': max_date.strftime('%Y-%m-%d')})])

In [796]:
nx.write_gexf(G=G, path='test.gexf')

[datetime.datetime(1931, 12, 13, 0, 0)]
[datetime.datetime(1932, 2, 6, 0, 0)]
[datetime.datetime(1932, 2, 26, 0, 0), datetime.datetime(1932, 3, 4, 0, 0), datetime.datetime(1932, 3, 5, 0, 0), datetime.datetime(1932, 3, 7, 0, 0), datetime.datetime(1932, 3, 11, 0, 0), datetime.datetime(1932, 3, 15, 0, 0)]
[datetime.datetime(1932, 9, 16, 0, 0)]
[datetime.datetime(1933, 2, 24, 0, 0)]
[datetime.datetime(1933, 11, 18, 0, 0)]
[datetime.datetime(1934, 1, 18, 0, 0), datetime.datetime(1934, 1, 20, 0, 0), datetime.datetime(1934, 1, 25, 0, 0), datetime.datetime(1934, 1, 27, 0, 0), datetime.datetime(1934, 1, 28, 0, 0), datetime.datetime(1934, 2, 1, 0, 0), datetime.datetime(1934, 2, 3, 0, 0), datetime.datetime(1934, 2, 5, 0, 0), datetime.datetime(1934, 2, 6, 0, 0), datetime.datetime(1934, 2, 7, 0, 0), datetime.datetime(1934, 2, 8, 0, 0)]
[datetime.datetime(1934, 3, 1, 0, 0)]
[datetime.datetime(1934, 5, 19, 0, 0), datetime.datetime(1934, 5, 24, 0, 0)]
[datetime.datetime(1934, 6, 28, 0, 0), datetime.da

In [381]:
for venue, count in venue_count.most_common():
    if count > 1:
        all_dates = sorted(list(set([x for x in dfs[venue]['Date']])))
        print(venue, len(all_dates), len(chunk_dates(all_dates, timedelta(days=14))))
        print()

K9 Club (Chicago, IL) 39 13

Echo Tavern (Albany, NY) 82 10

Kelly's Torch Club (Miami, FL) 26 3

Club Washington (East Liverpool, OH) 30 3

Torch Club (Miami, FL) 31 9

Ha-Ha Club (Miami, FL) 46 11

Club Frontenac (Detroit, MI) 55 20

The Bandbox (Cincinnati, OH) 22 3

Torch K9 Club (Miami, FL) 9 3

Club Richman (New York, NY) 69 35

Smith's Tavern (Albany, NY) 15 5

La Villa (Syracuse, NY) 17 5

Terrace Gardens (Warren, PA) 35 3

Club Top Hat (Detroit, MI) 14 6

Little Club (Saratoga Springs, NY) 11 4

Paddock (Miami, FL) 43 8

Finocchio's (San Francisco, CA) 31 30

Dwan's Recreation Gardens (Benton Harbor, MI) 10 3

Ellinee Village Inn (Benton Harbor, MI) 10 5

Cabin Inn (Chicago, IL) 15 15

Topper Cafe (Philadelphia, PA) 17 5

New Look Club (Cincinnati, OH) 13 6

Club Ambassador (Dayton, OH) 6 3

Riverside Grille (Hartford, CT) 7 3

Club Florida (New York, NY) 10 5

Band Box (Syracuse, NY) 10 1

Sabella's (New York, NY) 11 9

Joe’s Deluxe (Chicago, IL) 13 13

Bird's Cage (New York,

In [ ]:
dates = [y['all_dates'] for x,y in dfs.items()][0]

for period in periodize_dates(dates):
    for date in period:
        print(date.strftime('%Y-%m-%d'))

1931-12-13
1932-02-06
1932-02-26
1932-03-04
1932-03-05
1932-03-07
1932-03-11
1932-03-15
1932-09-16
1933-02-24
1933-11-18
1934-01-18
1934-01-20
1934-01-25
1934-01-27
1934-01-28
1934-02-01
1934-02-03
1934-02-05
1934-02-06
1934-02-07
1934-02-08
1934-03-01
1934-05-19
1934-05-24
1934-06-28
1934-06-29
1934-07-05
1934-07-12
1934-08-02
1934-08-06
1934-08-09
1934-09-06
1934-09-13
1934-10-17
1934-10-18
1934-10-25
1934-11-01
1934-12-13
1934-12-15
1934-12-20
1934-12-27
1935-01-03
1935-01-10
1935-04-14
1935-04-30
1935-06-02
1935-09-26
1935-10-01
1935-11-02
1935-11-05
1935-11-07
1935-11-09
1935-11-14
1935-12-07
1935-12-24
1936-02-08
1936-02-15
1936-02-22
1936-02-29
1936-03-07
1936-03-14
1936-03-21
1936-03-28
1936-04-04
1936-04-11
1936-04-15
1936-05-09
1944-12-06
